In [67]:
#Imports
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

import pandas as pd
import numpy as np

In [27]:
#settings
TRAIN_SZ = 0.7
VAL_SZ = 0.15
TEST_SZ = 0.15
FOLD_SZ = 4
LAGS_SZ = 6
H_FORECAST = 1
L_RATE = 0.005

In [28]:
#Data preprocessing

#Consumption dataframe
df_consumo = pd.read_csv(r"consumo_energia.csv")
df_consumo['date'] = pd.to_datetime(df_consumo['din_instante'])
df_consumo.set_index('date', inplace=True)
df_consumo.sort_index(inplace=True)
df_consumo.drop(['id_subsistema'],axis=1)

#Generation dataframe
df_geracao = pd.read_csv(r"geracao_energia.csv")
df_geracao['date'] = pd.to_datetime(df_geracao['index'])
df_geracao.set_index('date', inplace=True)
df_geracao['month'] = df_geracao.index.month
df_geracao['weekday'] = df_geracao.index.weekday
df_geracao.sort_index(inplace=True)
merged_df = pd.merge(df_geracao, df_consumo, on='date')

df = merged_df
df = df[['index', 'month', 'weekday', 'val_geracao', 'val_cargaenergiamwmed']]
train_set = df

#Z-Score Standarization
scaler = StandardScaler()
train_set['val_geracao_norm'] = scaler.fit_transform(train_set[['val_geracao']])
train_set['val_cargaenergiamwmed_norm'] = scaler.fit_transform(train_set[['val_cargaenergiamwmed']])
train_set['month_norm'] = scaler.fit_transform(train_set[['month']])
train_set['weekday_norm'] = scaler.fit_transform(train_set[['weekday']])

#defininf dependent and idependent variables
predictors = train_set[['month_norm','weekday_norm','val_geracao_norm','val_cargaenergiamwmed_norm']]
predicted = predictors[['val_geracao_norm']]


#Creating sequences
X, y = [], []
data_np = predictors.to_numpy()
pred_np = predicted.to_numpy()
for i in range(len(data_np) - LAGS_SZ - H_FORECAST + 1):
    X.append(data_np[i:i+LAGS_SZ])
    y.append(pred_np[i+LAGS_SZ:i+LAGS_SZ+H_FORECAST])

X = np.array(X)
y = np.array(y)

C:\Users\mateu\AppData\Local\Temp\ipykernel_17304\1839656982.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['val_geracao_norm'] = scaler.fit_transform(train_set[['val_geracao']])
C:\Users\mateu\AppData\Local\Temp\ipykernel_17304\1839656982.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['val_cargaenergiamwmed_norm'] = scaler.fit_transform(train_set[['val_cargaenergiamwmed']])
C:\Users\mateu\AppData\Local\Temp\ipykernel_17304\1839656982.py:27: SettingWithCopyWarning: 
A value i

In [ ]:
#Data set spliting
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]


X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_test_flat  = X_test.reshape((X_test.shape[0], -1))
y_train_flat = y_train[:, 0]
y_test_flat = y_test[:, 0]


In [63]:
#Stacking modeling
estimetors = [
    ('MLP', make_pipeline(StandardScaler(), MLPRegressor(alpha=1, max_iter=100))),
    ('SVR', make_pipeline(StandardScaler(),LinearSVR(random_state=42,max_iter=100))),
    ('SDG', make_pipeline(StandardScaler(),SGDRegressor()))
]
reg = StackingRegressor(estimators=estimetors, final_estimator=Ridge())

In [ ]:
#Stacking training
reg.fit(X_train_flat, y_train_flat)
score = reg.score(X_test_flat, y_test_flat)

c:\Users\mateu\OneDrive\Área de Trabalho\MESTRADO_2025\PPREVISÃO DE SERIES TEMPORAIS\SEL-for-Energy-Generation-Forecasting\sel\lib\site-packages\sklearn\ensemble\_stacking.py:1041: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\mateu\OneDrive\Área de Trabalho\MESTRADO_2025\PPREVISÃO DE SERIES TEMPORAIS\SEL-for-Energy-Generation-Forecasting\sel\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mateu\OneDrive\Área de Trabalho\MESTRADO_2025\PPREVISÃO DE SERIES TEMPORAIS\SEL-for-Energy-Generation-Forecasting\sel\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\mateu\OneDriv

In [ ]:
#Stacking and individual comparison
mlp_reg = make_pipeline(StandardScaler(), MLPRegressor(alpha=1, max_iter=100))
mlp_reg.fit(X_train_flat, y_train_flat)
score_mlp = mlp_reg.score(X_test_flat, y_test_flat)

svr_reg = make_pipeline(StandardScaler(),LinearSVR(random_state=42,max_iter=100))
svr_reg.fit(X_train_flat, y_train_flat)
score_svr = svr_reg.score(X_test_flat, y_test_flat)

sdg_reg = make_pipeline(StandardScaler(),SGDRegressor())
sdg_reg.fit(X_train_flat, y_train_flat)
score_sdg = sdg_reg.score(X_test_flat, y_test_flat)

print(f"Score MLP = {score_mlp}")
print(f"Score SVR = {score_svr}")
print(f"Score SDG = {score_sdg}")
print(f"Score Stacking = {score}")

c:\Users\mateu\OneDrive\Área de Trabalho\MESTRADO_2025\PPREVISÃO DE SERIES TEMPORAIS\SEL-for-Energy-Generation-Forecasting\sel\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1770: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Score MLP = 0.7310684480880978
Score SVR = 0.7194324966950493
Score SDG = 0.7297061796913545
Score Stacking = 0.7441126774300452


c:\Users\mateu\OneDrive\Área de Trabalho\MESTRADO_2025\PPREVISÃO DE SERIES TEMPORAIS\SEL-for-Energy-Generation-Forecasting\sel\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mateu\OneDrive\Área de Trabalho\MESTRADO_2025\PPREVISÃO DE SERIES TEMPORAIS\SEL-for-Energy-Generation-Forecasting\sel\lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\mateu\OneDrive\Área de Trabalho\MESTRADO_2025\PPREVISÃO DE SERIES TEMPORAIS\SEL-for-Energy-Generation-Forecasting\sel\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\mateu\OneDrive\